In [1]:
import os, re, time, sys
import pandas as pd

In [33]:
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = ["hundred", "thousand", "million", "billion", "trillion"]

      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
            break
#           raise Exception("Illegal word: " + word)
            

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current

In [41]:
def locFromText(text):
    t = re.search(r'Near (.*?),', text)
    if t:
        return t.group(1)
    else:
        None

In [4]:
df = pd.read_csv('allreports.txt', sep="/n", header=None )
df.rename(columns={0: "text"},inplace=True)
df.shape

/Users/andrewlb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


(2733, 1)

In [5]:
# Worried about dead values?
# df[df.text.map(len) > 10]

df.apply(lambda x : x["text"][x["text"].find("Near")],axis=1).unique()

array(['N', 'q'], dtype=object)

In [26]:
df.loc[29]

text        Near Al Hasakah, three airstrikes struck an ...
strikes    three airstrikes struck an ISIL large tactical...
Name: 29, dtype: object

In [34]:
# Get Number of Strikes
df["strikes"] = df.apply(lambda x: x["text"].split(' strike',1)[0].split(', ')[-1],axis=1)
df["strikes"] = df.apply(lambda x: x.strikes if x.strikes.isnumeric() else text2int(x.strikes), axis=1)
df["strikes"].head

<bound method NDFrame.head of 0        1
1        2
2       13
3        1
4        1
        ..
2728     5
2729     3
2730     1
2731     1
2732     1
Name: strikes, Length: 2733, dtype: object>

In [42]:
df["loc"] = df.apply(lambda x: locFromText(x.text), axis=1)
df.head

<bound method NDFrame.head of                                                    text strikes          loc
0      Near Al Shadaddi, one strike engaged an ISIS...       1  Al Shadaddi
1      Near Ar Raqqah, two strikes destroyed an ISI...       2    Ar Raqqah
2      Near Tabqah, 13 strikes engaged nine ISIS ta...      13       Tabqah
3      Near Al Qaim, one strike destroyed two anti-...       1      Al Qaim
4      Near Haditha, one strike engaged an ISIS tac...       1      Haditha
...                                                 ...     ...          ...
2728  • Near Bayji, five strikes engaged two ISIS ta...       5        Bayji
2729  • Near Huwijah, three strikes engaged two ISIS...       3      Huwijah
2730  • Near Rawah, one strike destroyed an ISIS VBI...       1        Rawah
2731  • Near Tuz, one strike engaged an ISIS tactica...       1          Tuz
2732  • Near Huwijah, one strike engaged an ISIS tac...       1      Huwijah

[2733 rows x 3 columns]>

In [47]:
df["loc"].unique()

array(['Al Shadaddi', 'Ar Raqqah', 'Tabqah', 'Al Qaim', 'Haditha',
       'Mosul', 'Tal Afar', 'Dayr Az Zawr', 'Raqqah', 'Huwijah', 'Rawah',
       'Abu Kamal', 'Fallujah', 'Hawijah', 'Al Hasakah', 'Kobani',
       'Bayji', 'Hit in Al Anbar Province', 'Sinjar', 'Mar’a', 'Palmyra',
       'Sultan Abdallah', 'Tal Abyad', 'Baghdadi', 'Al Huwayjah',
       'Makhmur', 'Ramadi', 'Aleppo', 'Al Asad', 'Kirkuk', 'Manbij',
       'Hit', 'Kisik', 'Qayyarah', 'Al Baghdadi', 'Ayn Isa', 'Habbaniyah',
       'Albu Hayat', 'At Tanf', 'Rutbah', 'Tuz', 'Hawayjah', 'Bashir',
       'Al Hawl', 'Al Hawijah', 'Tikrit', 'Ar Rutbah', 'Samarra', 'Idlib',
       'Huwayjah', 'Mahkmur', 'Washiyah', 'Habbaniya', 'Taji', 'Hajin',
       'Sulayman Bek', 'Tamakh', 'Erbil', 'al-Asad', 'al-Kisik',
       'Ash Shaddadi', 'Al Huwajah', 'Anbar Desert and Wadi-Ashai',
       'Makmur Mountains', 'Makmur', 'Lake Hamrin', 'Al Huwayja',
       'Hasakah', 'Day az Zawr', None,
       'Dayr Az Zawr; six strikes destroyed five ISI